<a href="https://colab.research.google.com/github/simbakm/training_yolov11_notebook/blob/main/examples/tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **starting Point @ simbakm Training YOlo  version 11 bold text**

# Setup

pip install `ultralytics` and [dependencies](https://github.com/ultralytics/ultralytics/blob/main/pyproject.toml) and check software and hardware.

[![PyPI - Version](https://img.shields.io/pypi/v/ultralytics?logo=pypi&logoColor=white)](https://pypi.org/project/ultralytics/) [![Downloads](https://static.pepy.tech/badge/ultralytics)](https://clickpy.clickhouse.com/dashboard/ultralytics) [![PyPI - Python Version](https://img.shields.io/pypi/pyversions/ultralytics?logo=python&logoColor=gold)](https://pypi.org/project/ultralytics/)

In [1]:
!uv pip install ultralytics
!pip install roboflow kagglehub
import ultralytics
import os
import shutil
from glob import glob
from tqdm import tqdm
import json
import random
from ultralytics import YOLO
ultralytics.checks()



Ultralytics 8.4.14 🚀 Python-3.12.12 torch-2.9.0+cu128 CUDA:0 (Tesla T4, 14913MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 41.9/112.6 GB disk)


**check for GPU**

In [2]:
import torch
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print(f"Using {device} device.")

Using cuda device.


**STEP 2 — Download the Roboflow dataset (YOLOv11-ready)bold text**

**helper fuction to couint the number of files**

In [22]:
import os
from collections import defaultdict

def count_file_types(dataset_path):
    file_counts = defaultdict(int)

    for root, dirs, files in os.walk(dataset_path):
        for file in files:
            ext = os.path.splitext(file)[1].lower()  # get file extension
            file_counts[ext] += 1

    print("File type counts:\n")
    for ext, count in sorted(file_counts.items()):
        print(f"{ext if ext else 'No Extension'} : {count}")

    return dict(file_counts)


In [27]:
from roboflow import Roboflow
rf = Roboflow(api_key="YdrZFjwWblcZBkXrqqaR")
project = rf.workspace("stairs-ttqqp").project("trash-bin-asn0s")
dataset = project.version(1).download("yolov11")

roboflow_path = dataset.location
print("Roboflow dataset:", roboflow_path)

counts = count_file_types(roboflow_path)



loading Roboflow workspace...
loading Roboflow project...
Roboflow dataset: /content/trash-bin-1
File type counts:

.jpg : 445
.txt : 447
.yaml : 1


## **STEP 3: Download the TACO dataset from KaggleHub**

In [5]:
import kagglehub
path = kagglehub.dataset_download("kneroma/tacotrashdataset")
print("Downloaded to:", path)
counts = count_file_types(path)


Using Colab cache for faster access to the 'tacotrashdataset' dataset.
Downloaded to: /kaggle/input/tacotrashdataset
File type counts:

.bin : 1
.csv : 1
.jpg : 1500
.json : 1
.txt : 1


**STEP 4 — Convert TACO (COCO JSON) → YOLO format**

In [6]:


# TACO dataset path (from KaggleHub)
taco_path = path
data_dir = os.path.join(taco_path, "data")

# Load annotations
annotations_path = os.path.join(data_dir, "annotations.json")
annotations = json.load(open(annotations_path))

# Output
output_taco = "/content/taco_yolo"
#delete privous derectories
if os.path.exists(output_taco):
    shutil.rmtree(output_taco)
    print("Old taco_yolo directory removed.")
else:
    print("No previous taco_yolo folder found.")

os.makedirs(f"{output_taco}/images", exist_ok=True)
os.makedirs(f"{output_taco}/labels", exist_ok=True)

# Map image id → image info
image_map = {img["id"]: img for img in annotations["images"]}

# Find all images recursively
all_images = {}
image_files = glob(f"{data_dir}/**/*.jpg", recursive=True) + \
              glob(f"{data_dir}/**/*.JPG", recursive=True)

for f in image_files:
    all_images[os.path.relpath(f, data_dir)] = f


print("Total images found:", len(all_images))

# Group annotations by image ID
ann_by_image = {}
for ann in annotations["annotations"]:
    img_id = ann["image_id"]
    ann_by_image.setdefault(img_id, []).append(ann)

# Convert to YOLO
for img_id, img_info in tqdm(image_map.items()):
    filename = img_info["file_name"]
    batch_relative_path = filename  # contains batch_x/000000.jpg

    if batch_relative_path not in all_images:
        print("Missing image:", batch_relative_path)
        continue

    src = all_images[batch_relative_path]

    # create unique new filename based on img_id
    new_name = f"{img_id}.jpg"
    dst = f"{output_taco}/images/{new_name}"

    shutil.copy(src, dst)

    # image width/height
    W = img_info["width"]
    H = img_info["height"]

    # label path (same name as image)
    label_path = f"{output_taco}/labels/{img_id}.txt"
    with open(label_path, "w") as f:
        for ann in ann_by_image.get(img_id, []):
            x, y, w, h = ann["bbox"]

            # convert to YOLO
            xc = (x + w/2) / W
            yc = (y + h/2) / H
            w /= W
            h /= H
            # Create category id mapping
            categories = annotations["categories"]
            cat_id_map = {cat["id"]: i for i, cat in enumerate(categories)}
            class_id = cat_id_map[ann["category_id"]]

            f.write(f"{class_id} {xc} {yc} {w} {h}\n")

print("DONE — TACO converted to YOLO:", output_taco)


No previous taco_yolo folder found.
Total images found: 1500


100%|██████████| 1500/1500 [00:42<00:00, 35.20it/s]

DONE — TACO converted to YOLO: /content/taco_yolo


# **5: Split the dataset into Train,val and test using ratio 80|10|10**

In [7]:
import os, random, shutil
from glob import glob

taco_img = "/content/taco_yolo/images"
taco_lbl = "/content/taco_yolo/labels"

images = sorted(glob(f"{taco_img}/*.jpg"))
random.shuffle(images)

n = len(images)
train_split = int(n * 0.8)
val_split = int(n * 0.9)

train_files = images[:train_split]
val_files   = images[train_split:val_split]
test_files  = images[val_split:]

def copy_split(files, folder):
    os.makedirs(f"/content/taco_split/images/{folder}", exist_ok=True)
    os.makedirs(f"/content/taco_split/labels/{folder}", exist_ok=True)
    for img in files:
        lbl = img.replace("images", "labels").replace(".jpg", ".txt")
        shutil.copy(img, f"/content/taco_split/images/{folder}/")
        shutil.copy(lbl, f"/content/taco_split/labels/{folder}/")

copy_split(train_files, "train")
copy_split(val_files, "val")
copy_split(test_files, "test")

print("TACO split complete!")


TACO split complete!


**Group TACO Classes into 10 & Shift IDs (COCO=0–79, TACO=80–89)**

In [23]:
taco_group_map = {
    0:0,1:0,2:0,3:0,      # Plastic -> class 0
    4:1,5:1,              # Glass -> class 1
    6:2,7:2,8:2,          # Metal -> class 2
    9:3,10:3,             # Paper/Cardboard -> class 3
    11:4,                  # Cigarette -> class 4
    12:5,13:5,14:5,       # Food wrapper -> class 5
    15:6,                  # Styrofoam -> class 6
    16:7,17:7,            # Rubber/Textile -> class 7
    18:8,19:8,            # Organic -> class 8
    20:9,21:9,22:9        # Electronics/Other -> class 9
}

# NO OFFSET - TACO classes will be 0-9
for split in ["train", "val", "test"]:
    lbl_files = glob.glob(f"/content/taco_split/labels/{split}/*.txt")
    for file in lbl_files:
        new_lines = []
        with open(file, "r") as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) > 0:
                    original_id = int(parts[0])
                    if original_id in taco_group_map:
                        grouped_id = taco_group_map[original_id]  # This gives 0-9
                        parts[0] = str(grouped_id)  # Direct mapping, no offset
                        new_lines.append(" ".join(parts))
        with open(file, "w") as f:
            f.write("\n".join(new_lines))

print("TACO grouped into 10 classes (classes 0-9):")
class_names = ["plastic", "glass", "metal", "paper", "cigarette",
               "food_wrapper", "styrofoam", "rubber_textile", "organic", "electronics_other"]
for i, name in enumerate(class_names):
    print(f"  Class {i}: {name}")

TACO grouped into 10 classes (classes 0-9):
  Class 0: plastic
  Class 1: glass
  Class 2: metal
  Class 3: paper
  Class 4: cigarette
  Class 5: food_wrapper
  Class 6: styrofoam
  Class 7: rubber_textile
  Class 8: organic
  Class 9: electronics_other


**STEP 6 — move trans bin to class 90**

In [31]:
import glob

rf_labels = glob.glob(f"{roboflow_path}/**/labels/*.txt", recursive=True)

for file in rf_labels:
    lines = []
    with open(file, "r") as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) > 0:
                parts[0] = "10"  # Move to class 10 (trash bin)
                lines.append(" ".join(parts))

    if lines:  # Only write if there are lines
        with open(file, "w") as f:
            f.write("\n".join(lines))

print(f"Roboflow labels shifted to class 10 (trash bin). Processed {len(rf_labels)} files.")

Roboflow labels shifted to class 10 (trash bin). Processed 445 files.


**Create a combined Dataset**

In [32]:
import os
import shutil
import glob
import random

# Remove old combined folder
shutil.rmtree("/content/combined", ignore_errors=True)

# Create combined structure
for split in ["train", "val", "test"]:
    os.makedirs(f"/content/combined/images/{split}", exist_ok=True)
    os.makedirs(f"/content/combined/labels/{split}", exist_ok=True)

# Copy TACO dataset (classes 0-9)
print("Copying TACO dataset (classes 0-9)...")
taco_dirs = {
    'train': '/content/taco_split/images/train',
    'val': '/content/taco_split/images/val',
    'test': '/content/taco_split/images/test'
}

taco_counts = {}
for split, path in taco_dirs.items():
    if os.path.exists(path):
        # Copy images
        !cp -r {path}/* /content/combined/images/{split}/ 2>/dev/null

        # Copy labels
        label_path = path.replace('images', 'labels')
        if os.path.exists(label_path):
            !cp -r {label_path}/* /content/combined/labels/{split}/ 2>/dev/null

        # Count copied items
        img_count = len(glob.glob(f"/content/combined/images/{split}/*.jpg"))
        taco_counts[split] = img_count
        print(f"  Copied {img_count} TACO images to {split} (classes 0-9)")

# Copy Roboflow dataset (class 10 - trash bin)
print("\nCopying Roboflow dataset (class 10 - trash bin)...")


if os.path.exists(roboflow_path):
    roboflow_counts = {}
    for split in ['train', 'valid', 'test']:
        target_split = 'val' if split == 'valid' else split

        # Copy images
        img_source = f"{roboflow_path}/{split}/images"
        if os.path.exists(img_source):
            !cp -r {img_source}/* /content/combined/images/{target_split}/ 2>/dev/null

        # Copy labels
        label_source = f"{roboflow_path}/{split}/labels"
        if os.path.exists(label_source):
            !cp -r {label_source}/* /content/combined/labels/{target_split}/ 2>/dev/null

        # Count copied items
        img_count = len(glob.glob(f"/content/combined/images/{target_split}/*.jpg"))
        roboflow_counts[target_split] = img_count
        print(f"  Copied {img_count} Roboflow images to {target_split} (class 10)")
else:
    print(f"  Warning: Roboflow path {roboflow_path} does not exist!")
    print("  Please update the roboflow_path variable with the correct path.")

# Final verification
print("\n" + "="*60)
print("FINAL COMBINED DATASET COUNTS (11 total classes: 0-10)")
print("="*60)

total_images = 0
total_labels = 0

for split in ["train", "val", "test"]:
    images = glob.glob(f"/content/combined/images/{split}/*.jpg")
    labels = glob.glob(f"/content/combined/labels/{split}/*.txt")
    print(f"{split.upper():5} - Images: {len(images):6d}, Labels: {len(labels):6d}")
    total_images += len(images)
    total_labels += len(labels)

    # Check for missing labels
    missing = 0
    for img in images:
        label = img.replace('/images/', '/labels/').replace('.jpg', '.txt')
        if not os.path.exists(label):
            missing += 1
    if missing > 0:
        print(f"         ⚠️  Warning: {missing} images missing labels in {split}")
    else:
        print(f"         ✅ All images have corresponding labels in {split}")

    # Check class distribution in this split
    class_counts = {i:0 for i in range(11)}  # Initialize all 11 classes
    for label_file in labels:
        with open(label_file, 'r') as f:
            for line in f:
                if line.strip():
                    class_id = int(line.strip().split()[0])
                    if class_id in class_counts:
                        class_counts[class_id] += 1

    print(f"         Class distribution in {split}:")
    class_names = ["plastic", "glass", "metal", "paper", "cigarette",
                   "food_wrapper", "styrofoam", "rubber_textile", "organic",
                   "electronics_other", "trash_bin"]
    for i in range(11):
        if class_counts[i] > 0:
            print(f"           Class {i} ({class_names[i]}): {class_counts[i]}")

print("="*60)
print(f"TOTAL - Images: {total_images}, Labels: {total_labels}")
print("="*60)

Copying TACO dataset (classes 0-9)...
  Copied 1200 TACO images to train (classes 0-9)
  Copied 150 TACO images to val (classes 0-9)
  Copied 150 TACO images to test (classes 0-9)

Copying Roboflow dataset (class 10 - trash bin)...
  Copied 1514 Roboflow images to train (class 10)
  Copied 255 Roboflow images to val (class 10)
  Copied 176 Roboflow images to test (class 10)

FINAL COMBINED DATASET COUNTS (11 total classes: 0-10)
TRAIN - Images:   1514, Labels:   1514
         ✅ All images have corresponding labels in train
         Class distribution in train:
           Class 10 (trash_bin): 398
VAL   - Images:    255, Labels:    255
         ✅ All images have corresponding labels in val
         Class distribution in val:
           Class 10 (trash_bin): 139
TEST  - Images:    176, Labels:    176
         ✅ All images have corresponding labels in test
         Class distribution in test:
           Class 10 (trash_bin): 32
TOTAL - Images: 1945, Labels: 1945


count the images in each category -> train,val and test

**step7: Create YOLO data.yaml**

In [33]:
from ultralytics import YOLO

# Define our 11 classes (TACO 0-9 + Trash Bin 10)
final_names = [
    # TACO grouped classes (10 classes) - classes 0-9
    "plastic",           # class 0
    "glass",             # class 1
    "metal",             # class 2
    "paper",             # class 3
    "cigarette",         # class 4
    "food_wrapper",      # class 5
    "styrofoam",         # class 6
    "rubber_textile",    # class 7
    "organic",           # class 8
    "electronics_other", # class 9

    # Roboflow class (1 class) - class 10
    "trash_bin"          # class 10
]

# Create combined.yaml
with open("combined.yaml", "w") as f:
    f.write("# TACO (10 classes: 0-9) + Trash Bin (1 class: 10) dataset\n\n")
    f.write("train: /content/combined/images/train\n")
    f.write("val: /content/combined/images/val\n")
    f.write("test: /content/combined/images/test\n\n")
    f.write(f"nc: {len(final_names)}  # 11 classes total (0-10)\n")
    f.write("names:\n")
    for name in final_names:
        f.write(f"  - {name}\n")

print("✅ combined.yaml created with 11 classes (0-10):")
for i, name in enumerate(final_names):
    print(f"   Class {i}: {name}")

# Verify the paths exist
print("\nVerifying paths:")
for split in ["train", "val", "test"]:
    path = f"/content/combined/images/{split}"
    if os.path.exists(path):
        count = len(glob.glob(f"{path}/*.jpg"))
        print(f"  {split}: {path} - {count} images ✅")
    else:
        print(f"  {split}: {path} - NOT FOUND ❌")

✅ combined.yaml created with 11 classes (0-10):
   Class 0: plastic
   Class 1: glass
   Class 2: metal
   Class 3: paper
   Class 4: cigarette
   Class 5: food_wrapper
   Class 6: styrofoam
   Class 7: rubber_textile
   Class 8: organic
   Class 9: electronics_other
   Class 10: trash_bin

Verifying paths:
  train: /content/combined/images/train - 1514 images ✅
  val: /content/combined/images/val - 255 images ✅
  test: /content/combined/images/test - 176 images ✅


# **8: — Train YOLOv11**

In [35]:
from ultralytics import YOLO

# Path to your combined YAML
data_yaml = "combined.yaml"

# Use YOLO11n pretrained on COCO as base
model = YOLO("yolo11n.pt")

# Training parameters - saving to default path /content/runs/detect/train/
model.train(
    data=data_yaml,        # 11-class dataset (0-10)
    epochs=100,            # max epochs
    imgsz=640,             # image size
    batch=16,              # adjust to GPU memory
    device=device,         # device is already defined in previous cells
    patience=10,           # early stopping if val mAP not improving for 10 epochs
    save=True,             # saves best.pt automatically
    pretrained=True,       # start from COCO weights
    optimizer='auto',      # YOLO chooses best optimizer
    lr0=0.01,              # initial learning rate
    augment=True,          # enables basic augmentations

    # Augmentation parameters
    mosaic=1.0,            # mosaic augmentation (0-1 probability)
    mixup=0.1,             # mixup augmentation (0-1 probability)
    flipud=0.0,            # vertical flip - 0.0 as trash isn't usually upside down
    fliplr=0.5,            # horizontal flip - useful for symmetry
    hsv_h=0.015,           # hue jitter
    hsv_s=0.7,             # saturation jitter
    hsv_v=0.4,             # value (brightness) jitter
    degrees=0.0,           # rotation (0 for no rotation - keeps annotations valid)
    translate=0.1,         # translation
    scale=0.5,             # scaling
    shear=0.0,             # shear (keep 0 to avoid distortion)

    # Training settings
    deterministic=True,    # reproducible results
    save_period=5,         # save weights every 5 epochs
    plots=True,            # generate loss/metrics plots automatically
    workers=8,             # number of dataloader workers
    seed=42,               # for reproducibility
    verbose=True           # print detailed output
    # project and name are removed - will use default: runs/detect/train/
)


engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=True, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=combined.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.1, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=train4, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=10, perspective=0.0, plots=True, pose=12.0, pretrained=True, profile=False, project=None, rect=False, resume=False

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([10])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x793f6a1c97c0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048,

## **STEP 6 — Download trained model**

In [1]:
from google.colab import files
files.download("/content/runs/detect/train4/weights/best.pt")


FileNotFoundError: Cannot find file: /content/runs/detect/train4/weights/best.pt